# Thai G2P with Seq2Seq model

Code by @artificiala https://github.com/artificiala

In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [2]:
#cd /content/gdrive/My\ Drive/

In [3]:
#mkdir g2pthai

In [4]:
#cd g2pthai

In [5]:
%pwd

'D:\\nlp\\g2p\\new'

In [6]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals


In [7]:
import time
import sys
import os
import random
import math
import traceback
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms, utils
import numpy as np
from tqdm.auto import tqdm
from collections import Counter
from matplotlib import pyplot as plt
from collections import OrderedDict

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


SEED = 0
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [8]:
import os
from matplotlib import font_manager as fm, rcParams

fpath = "./Loma-Bold.ttf"#os.path.join(rcParams["datapath"], "")
prop = fm.FontProperties(fname=fpath)
# ax.set_title('This is a special font: {}'.format(fname), fontproperties=prop)
prop

In [9]:
#!nvidia-smi

In [10]:
# Check if GPUs are in the machine, otherwise assign device as CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [11]:
#!git clone https://github.com/artificiala/thai-romanization.git

In [12]:
#!ls

In [13]:
#!wget https://github.com/PyThaiNLP/lexicon-thai/raw/master/G2P/wiktionary-11-2-2020.tsv

The csv file contains two columns indicates Thai text and its corresponding English tranliteration

In [14]:
DATA_PATH = './wiktionary-11-2-2020.tsv'
#!head $DATA_PATH

In [15]:
# number fo lines
#!wc -l $DATA_PATH

In [16]:
def load_data(data_path):
    with open(data_path, 'r', encoding='utf-8-sig') as f:
        lines = f.read().rstrip().split('\n')

    input_texts = []
    target_texts = []
    for index, line in enumerate(lines):
        line = line#.replace(u'\xa0',' ')
        try:
            input_text, target_text = line.split("	")
            input_texts.append(input_text)
            target_texts.append(target_text)
        except Exception as e:
            print(e)
            print('line [{}]:{}'.format(index+1, line))
            # traceback.print_exc()
            continue
    return input_texts, target_texts


In [17]:
# extract
input_texts, target_texts = load_data(DATA_PATH)


In [18]:
print(input_texts[0])
print(target_texts[0])

ก
k ɔː ˧


In [19]:
print(len(input_texts))

14483


In [20]:
src_len_counter_in = Counter()
trg_len_counter_in = Counter()

for input in input_texts:
    src_len_counter_in[len(input)] += 1
for input in target_texts:
    trg_len_counter_in[len(input)] += 1

print(src_len_counter_in)
print(trg_len_counter_in)

Counter({6: 2136, 4: 1820, 7: 1803, 5: 1775, 3: 1600, 8: 1381, 9: 1092, 10: 768, 11: 484, 12: 377, 2: 346, 13: 220, 14: 184, 15: 109, 16: 83, 17: 64, 1: 51, 18: 49, 20: 30, 19: 28, 21: 19, 22: 17, 23: 12, 25: 6, 24: 4, 26: 4, 28: 3, 31: 3, 29: 3, 36: 2, 35: 2, 45: 2, 33: 2, 27: 1, 32: 1, 40: 1, 30: 1})
Counter({20: 892, 19: 855, 21: 809, 18: 784, 8: 754, 9: 747, 10: 671, 22: 620, 17: 493, 23: 470, 30: 413, 11: 413, 29: 361, 7: 356, 31: 348, 32: 334, 24: 319, 28: 301, 27: 293, 33: 276, 25: 246, 26: 245, 12: 244, 16: 222, 34: 211, 35: 182, 39: 164, 37: 161, 36: 159, 38: 158, 15: 132, 41: 131, 40: 120, 42: 119, 44: 116, 43: 112, 6: 110, 45: 82, 13: 80, 46: 78, 48: 76, 47: 68, 49: 58, 51: 41, 50: 40, 54: 39, 55: 38, 56: 38, 52: 36, 59: 35, 60: 34, 53: 33, 57: 26, 58: 26, 14: 23, 68: 20, 63: 20, 61: 20, 66: 17, 62: 17, 69: 14, 64: 12, 65: 11, 67: 10, 70: 10, 73: 10, 75: 9, 79: 9, 81: 9, 71: 9, 72: 9, 82: 8, 74: 7, 77: 6, 80: 5, 83: 4, 76: 4, 78: 4, 84: 3, 87: 3, 98: 3, 165: 3, 88: 2, 95: 2,

In [21]:
# Define special characters
UNK_token = '<UNK>'
PAD_token = '<PAD>'
START_token = '<start>'
END_token = '<end>'
MAX_LENGTH = 70

class Language:
    def __init__(self, name, is_input=False):
        self.name = name
        self.characters = set()
        self.n_chars = 0
        self.char2index = {}
        self.index2char = {}

        if is_input == True:
            self.index2char = { 0: PAD_token, 1: UNK_token, 2: START_token, 3: END_token }
            self.char2index = { ch:i for i, ch in self.index2char.items() } #reverse dictionary
            self.n_chars = 4
        else:
            self.index2char = { 0: PAD_token, 1: START_token, 2: END_token }
            self.char2index = { ch:i for i, ch in self.index2char.items() } #reverse dictionary
            self.n_chars = 3

    def addText(self, text):
        for character in text:
            self.addCharacter(character)
    
    def addCharacter(self, character):
        if character not in self.char2index.keys():
            self.char2index[character] = self.n_chars
            self.index2char[self.n_chars] = character
            self.n_chars += 1
            
            
def indexesFromText(lang, text):
    """returns indexes for all character given the text in the specified language"""
    return [lang.char2index[char] for char in text]

def tensorFromText(lang, text):
    """construct a tensor given the text in the specified language"""
    indexes = indexesFromText(lang, text)
    indexes.append(lang.char2index[END_token])
    
    no_padded_seq_length = len(indexes) # Number of characters in the text (including <END> token)
    # Add padding token to make all tensors in the same length
    for i in range(len(indexes), MAX_LENGTH): # padding
        indexes.append(lang.char2index[PAD_token])
        
    return torch.tensor(indexes, dtype=torch.long).to(device), no_padded_seq_length

def filterPair(p1, p2):
    """filter for the pair the both texts has length less than `MAX_LENGTH`"""
    return len(p1) < MAX_LENGTH and len(p2) < MAX_LENGTH

def tensorsFromPair(pair, lang1, lang2):
    """construct two tensors from a pair of source and target text specified by source and target language"""
    input_tensor, input_length = tensorFromText(lang1, pair[0])
    target_tensor, target_length = tensorFromText(lang2, pair[1])
    return input_tensor, target_tensor, input_length, target_length



class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        input_text, target_text, lang_th, lang_th_romanized = sample['input_text'], sample['target_text'], \
                                                              sample['lang_th'], sample['lang_th_romanized']

        input_tensor, target_tensor, input_length, target_length = tensorsFromPair([input_text, target_text], 
                                                                                   lang_th, 
                                                                                   lang_th_romanized)
        
        return {
                'input_text': input_text,
                'target_text': target_text,
                'input_length': input_length,
                'target_length': target_length,
                'input_tensor': input_tensor,
                'target_tensor': target_tensor
               }
    
    
class ThaiRomanizationDataset(Dataset):
    """Thai Romanization Dataset class"""
    def __init__(self, 
                 data_path=DATA_PATH, 
                 transform=transforms.Compose([ ToTensor() ])):

        input_texts, target_texts = load_data(data_path)
        
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.transform = transform
        self.lang_th = None
        self.lang_th_romanized = None
        self.counter = Counter()
        self.pairs = []
        self.prepareData()

    def prepareData(self):
        self.lang_th = Language('th', is_input=True)
        self.lang_th_romanized = Language('th_romanized', is_input=False)
        for i in range(len(self.input_texts)):
            
            input_text = self.input_texts[i]
            target_text = self.target_texts[i]
            
            # Count the number of input and target sequences with length `x`
            self.counter.update({ 
                                  'len_input_{}'.format(len(input_text)): 1, 
                                  'len_target_{}'.format(len(target_text)): 1 
                                })
            
            if filterPair(input_text, target_text):
                self.pairs.append((input_text, target_text))
                self.lang_th.addText(input_text)
                self.lang_th_romanized.addText(target_text)    

    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        
        sample = dict()
        sample['input_text'] = self.pairs[idx][0]
        sample['target_text'] = self.pairs[idx][1]
        
        sample['lang_th'] = self.lang_th
        sample['lang_th_romanized'] = self.lang_th_romanized

        if self.transform:
            sample = self.transform(sample)

        return sample
    

def save_model(name, epoch, loss, model):
    print('Save model at epoch ', epoch)
    torch.save({
        'epoch': epoch,
        'loss': loss,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'char_to_ix': thai_romanization_dataset.lang_th.char2index,
        'ix_to_char': thai_romanization_dataset.lang_th.index2char,
        'target_char_to_ix': thai_romanization_dataset.lang_th_romanized.char2index,
        'ix_to_target_char':thai_romanization_dataset.lang_th_romanized.index2char,
        'encoder_params': (INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT), 
        'decoder_params': (OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DEC_DROPOUT)
        
    }, "{}.best_epoch-{}.tar".format(name, epoch))
    
 
    
def load_model(model_path):
    
    data = torch.load(model_path)
    
    INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT = data['encoder_params']
    OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DEC_DROPOUT = data['decoder_params']

    
    encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM).to(device)
    decoder = AttentionDecoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM).to(device)

    model = Seq2Seq(encoder, decoder).to(device)
    
    model.load_state_dict(data['model_state_dict'])
    
    
    learning_rate = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer.load_state_dict(data['optimizer_state_dict'])
    criterion = nn.CrossEntropyLoss(ignore_index = 0)

    char_to_ix = data['char_to_ix']
    ix_to_char = data['ix_to_char'] 
    target_char_to_ix = data['target_char_to_ix']
    ix_to_target_char = data['ix_to_target_char']
    
    
    return {
        'model': model,
        'optmizer': optimizer,
        'char_to_ix': char_to_ix,
        'ix_to_char' : ix_to_char,
        'target_char_to_ix': target_char_to_ix,
        'ix_to_target_char': ix_to_target_char
    }
       

In [22]:
thai_romanization_dataset = ThaiRomanizationDataset()

In [23]:
thai_romanization_dataset.lang_th_romanized.index2char

{0: '<PAD>',
 1: '<start>',
 2: '<end>',
 3: 'k',
 4: ' ',
 5: 'ɔ',
 6: 'ː',
 7: '˧',
 8: '.',
 9: 'ʰ',
 10: 't',
 11: 'm',
 12: 'p',
 13: 'j',
 14: 'o',
 15: '̚',
 16: '˨',
 17: '˩',
 18: 'a',
 19: 'u',
 20: '˦',
 21: '˥',
 22: 'n',
 23: 'ŋ',
 24: 'l',
 25: 's',
 26: 'i',
 27: 'w',
 28: '̯',
 29: 'b',
 30: '͡',
 31: 'ɕ',
 32: 'r',
 33: 'h',
 34: 'e',
 35: 'ʔ',
 36: 'd',
 37: 'ɯ',
 38: 'f',
 39: 'ɤ',
 40: 'ɛ'}

## Seq2Seq Model architecture

## 1. Encoder

Encoder 
    - Embedding layer :(vocaburay_size, embedding_size) 
        Input: (batch_size, sequence_length)
        Output: (batch_size, sequence_length, embebeding_size)
      
    - Bi-LSTM layer : (input_size, hidden_size, num_layers, batch_first=True)
        Input: (input=(batch_size, seq_len, embebeding_size),  hidden)
        Output: (output=(batch_size, seq_len, hidden_size),
                 (h_n, c_n))
     
     
__Steps:__

1. Receives a batch of source sequences (batch_size, MAX_LENGTH) and a 1-D array of the length for each sequence (batch_size).
     
2. Sort sequences in the batch by sequence length (number of tokens in the sequence where <PAD> token is excluded).

3. Feed the batch of sorted sequences into the Embedding Layer to maps source character indices into vectors. (batch_size,  sequence_length, embebeding_size)

4. Use `pack_padded_sequence` to let LSTM packed input with same length at time step $t$ together. This will reduce time required for training by avoid feeding `<PAD>` token to the LSTMs.


5. Returns LSTM outputs in the unsorted order, and the LSTM hidden state vectors.
     

In [24]:


class Encoder(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_size, hidden_size, dropout=0.5):
        """Constructor"""
        super(Encoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.character_embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.rnn = nn.LSTM(input_size=embedding_size, 
                            hidden_size=hidden_size // 2, 
                            bidirectional=True,
                            batch_first=True)
        
        self.dropout = nn.Dropout(dropout)


    def forward(self, sequences, sequences_lengths):
        batch_size = sequences.size(0)
        hidden = self.init_hidden(batch_size) # batch_size

#         print('encoder, hidden:', hidden)
        # sequences :(batch_size, sequence_length=MAX_LENGTH)
        # sequences_lengths: (batch_size)  # an 1-D indicating length of each sequence (excluded <PAD> token) in `seq`
        
        # 1. Firstly we sort `sequences_lengths` according to theirs values and keep list of indexes to perform sorting
        sequences_lengths = np.sort(sequences_lengths)[::-1] # sort in ascending order and reverse it
        index_sorted = np.argsort(-sequences_lengths) # use negation in sort in descending order
        index_unsort = np.argsort(index_sorted) # to unsorted sequence
        
        
        # 2. Then, we change position of sequence in `sequences` according to `index_sorted`
        index_sorted = torch.from_numpy(index_sorted)
        sequences = sequences.index_select(0, index_sorted.to(device))
        
        # 3. Feed to Embedding Layer
        
        sequences = self.character_embedding(sequences)
        sequences = self.dropout(sequences)
        
#         print('sequences',sequences.size(), sequences)
            
        # 3. Use function: pack_padded_sequence to let LSTM packed input with same length at time step T together
        
        # Quick fix: Use seq_len.copy(), instead of seq_len to fix `Torch.from_numpy not support negative strides`
        # ndarray.copy() will alocate new memory for numpy array which make it normal, I mean the stride is not negative any more.

        sequences_packed = nn.utils.rnn.pack_padded_sequence(sequences,
                                                             sequences_lengths.copy(),
                                                             batch_first=True)
#         print('sequences_packed', sequences_packed)

        # 4. Feed to LSTM
        sequences_output, hidden = self.rnn(sequences_packed, hidden)
        
        # 5. Unpack
        sequences_output, _ = nn.utils.rnn.pad_packed_sequence(sequences_output, batch_first=True)

        # 6. Un-sort by length
        index_unsort = torch.from_numpy(index_unsort).to(device)
        sequences_output = sequences_output.index_select(0, Variable(index_unsort))

#         print('hidden shape', self.hidden[0].shape, self.hidden[0], self.hidden[1].shape, self.hidden[1])
        return sequences_output, hidden
    
    def init_hidden(self, batch_size):
        h_0 = torch.zeros([2, batch_size, self.hidden_size // 2], requires_grad=True).to(device)
        c_0 = torch.zeros([2, batch_size, self.hidden_size // 2], requires_grad=True).to(device)

        return (h_0, c_0)
    

## Test encoder

In [25]:
### Testing Encoder part
# TODO: put whether GPU is available or not
# Device

INPUT_DIM = len(thai_romanization_dataset.lang_th.char2index)
OUTPUT_DIM = len(thai_romanization_dataset.lang_th_romanized.char2index)

ENC_EMB_DIM = 10
ENC_HID_DIM = 10
ENC_DROPOUT = 0.5

_encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM).to(device)

# obtain one sample from the data iterator

## Decoder

   
Decoder architecture

    - Embedding layer :(vocabulary_size, embebeding_size)
        Input: (batch_size, sequence_length=1)
        Output: (batch_size, sequence_length=1, embebeding_size)
    - RNN layer :input_size=embebeding_size, hidden_size, num_layers, batch_first=True)
        Input: (input=(batch_size, input_size=embedding_dimension), hidden:tuple=encoder_hidden
        Output: (batch_size, seq_len, hidden_size), (h_n, c_n)
    - Attention Layer: (in_features=hidden_size, out_features=hidden_size, bias=True)
    - Linear Layer: (in_features, out_features=vocabulary_size)
        Input: (batch_size, hidden_size)
        Output: (batch_size, vocabulary_size)
    
    - Softmax layer
        Input: (batch_size, vocabulary_size)
        Output: (batch_size, vocabulary_size)



For the Attention mechanishm in the Decoder, Luong-style attention [[Luong et. al (2015)](https://arxiv.org/abs/1508.04025)] is used. 



__Steps:__

1. Receives a batch of <start> token (batch_size, 1) and a batch of Encoder's hidden state.
     
2. Embed input into vectors.

3. Feed vectors from (2) to the LSTM.

4. Feed the output of LSTM at time step $t_1$ and Encoder output to the Attention Layer.

5. Attention layer, returns weights for Encoder's hidden states in every time step (masked out the time step with <PAD> token), then multiply with Encoder's hidden states to obtain a context vector
    
6. Concatenate both decoder hidden state and the context vector, feed to a linear layer, and return its output.

7. Decoder then returns, final output, decoder's hidden state, attention weights, and context vector at time step $t$

In [26]:

class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()

        self.method = method
        self.hidden_size = hidden_size

        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, self.hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, self.hidden_size)
            self.other = nn.Parameter(torch.FloatTensor(1, self.hidden_size))

    def forward(self, hidden, encoder_outputs, mask):
        # hidden: B x 1 x h ; 
        # encoder_outputs: B x S x h

        # Calculate energies for each encoder output
        if self.method == 'dot':
            attn_energies = torch.bmm(encoder_outputs, hidden.transpose(1, 2)).squeeze(2)  # B x S
        elif self.method == 'general':
            attn_energies = self.attn(encoder_outputs.view(-1, encoder_outputs.size(-1)))  # (B * S) x h
            attn_energies = torch.bmm(attn_energies.view(*encoder_outputs.size()),
                                      hidden.transpose(1, 2)).squeeze(2)  # B x S
        elif self.method == 'concat':
            attn_energies = self.attn(
                torch.cat((hidden.expand(*encoder_outputs.size()), encoder_outputs), 2))  # B x S x h
            attn_energies = torch.bmm(attn_energies,
                                      self.other.unsqueeze(0).expand(*hidden.size()).transpose(1, 2)).squeeze(2)

        attn_energies = attn_energies.masked_fill(mask == 0, -1e10)

        # Normalize energies to weights in range 0 to 1
        return F.softmax(attn_energies, 1)

class AttentionDecoder(nn.Module): 
    
    def __init__(self, vocabulary_size, embedding_size, hidden_size, dropout=0.5):
        """Constructor"""
        super(AttentionDecoder, self).__init__()
        self.vocabulary_size = vocabulary_size
        self.hidden_size = hidden_size
        self.character_embedding = nn.Embedding(vocabulary_size, embedding_size)
        self.rnn = nn.LSTM(input_size=hidden_size + embedding_size,
                            hidden_size=hidden_size,
                            bidirectional=False,
                            batch_first=True)
        
        self.attn = Attn(method="general", hidden_size=self.hidden_size)
        self.linear = nn.Linear( hidden_size, vocabulary_size)
        
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, input, last_hidden, encoder_outputs, mask):
        """"Defines the forward computation of the decoder"""
        # input: (B, 1) ,
        # last_hidden: (num_layers * num_directions = 2, B, hidden_dim)
        # encoder_outputs: (B, S, hidden_dim)
        
        # last_hidden from  (batch_size, hidden size)
        # to (batch_size, 1, hidden size)
        hidden = last_hidden.permute(1, 0, 2)
#         print('unsqueeze hidden.size()', hidden.size())

        attn_weights = self.attn(hidden, encoder_outputs, mask)  # B x S

        #  context = (B, 1, S) x (B, S, hidden_dim)
        #  context = (B, 1, hidden_dim)
        context_vector = attn_weights.unsqueeze(1).bmm(encoder_outputs)  
        #  context after sum = (B, hidden_dim)
        context_vector = torch.sum(context_vector, dim=1)
        
        
        embedded = self.character_embedding(input)
        embedded = self.dropout(embedded)
        
        # context_vector = (B, hidden_dim)
        # embedded = (B, emb_dim)
        # rnn_input = (B, hidden_dim + emb_dim)
        context_vector = context_vector.unsqueeze(1)
        
        # print('context_vector', context_vector.size())
        # print('embedded', embedded.size())
        rnn_input = torch.cat((context_vector, embedded), -1)
    
    
        # output (batch_size, 1, hidden_dim)
        output, hidden = self.rnn(rnn_input)        
        
#         print('hidden[0].size()', hidden[0].size(), hidden[0], )
        output =  output.view(-1, output.size(2))
        
        x = self.linear(output)
#         print('decoder hidden[0]', hidden[0].size(), hidden[0])
        return x, hidden[0], attn_weights

    def init_hidden(self, batch_size):
        h_0 = torch.zeros([1, batch_size, self.hidden_size ], requires_grad=True).to(device)
        c_0 = torch.zeros([1, batch_size, self.hidden_size ], requires_grad=True).to(device)

        return (h_0, c_0)

## Seq2Seq model

This class encapsulate _Decoder_ and _Encoder_ class.

__Steps:__

1. The input sequcence $X$ is fed into the encoder to receive one hidden state vector.

2. The initial decoder hidden state is set to be the hidden state vector of the encoder

3. Add a batch of `<start>` tokens (batch_size, 1) as the first input $y_1$
    
4. Then, decode within a loop:
    - Inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector $z$ into the decoder
    - Receiveing a prediction $\hat{y}$ and a new hidden state $s_t$
    - Then, either use teacher forcing to let groundtruth target character as the input for the decoder at time step $t+1$, or let the result from decoder as the input for the next time step.

In [27]:
class Seq2Seq(nn.Module): 

    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = 0

        assert encoder.hidden_size == decoder.hidden_size
    
    def create_mask(self, source_seq):
        mask = (source_seq != self.pad_idx)
        return mask
        
  
    def forward(self, source_seq, source_seq_len, target_seq, teacher_forcing_ratio = 0.5):
        """
            Parameters:
                - source_seq: (batch_size x MAX_LENGTH) 
                - source_seq_len: (batch_size x 1)
                - target_seq: (batch_size x MAX_LENGTH)

            Returns
        """
        batch_size = source_seq.size(0)
        start_token = thai_romanization_dataset.lang_th_romanized.char2index["<start>"]
        end_token = thai_romanization_dataset.lang_th_romanized.char2index["<end>"]
        max_len = MAX_LENGTH
        target_vocab_size = self.decoder.vocabulary_size

        # init a tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(device)
        
        if target_seq is None:
            assert teacher_forcing_ratio == 0, "Must be zero during inference"
            inference = True
        else:
            inference = False

    
        # feed mini-batch source sequences into the `Encoder`
        encoder_outputs, encoder_hidden = self.encoder(source_seq, source_seq_len)

        # create a Tensor of first input for the decoder
        decoder_input = torch.tensor([[start_token] * batch_size]).view(batch_size, 1).to(device)
         
        encoder_hidden_ht = torch.cat([encoder_hidden[0][0], encoder_hidden[0][1]], dim=1).unsqueeze(dim=0)
        decoder_hidden = encoder_hidden_ht

            
#         print('decoder_hidden', decoder_hidden.size())
        # define a context vector
#         decoder_context = Variable(torch.zeros(encoder_outputs.size(0), encoder_outputs.size(2))).unsqueeze(1).to(device)
        
        
        max_source_len = encoder_outputs.size(1)
        mask = self.create_mask(source_seq[:, 0:max_source_len])
            
        
        attn_weights  = []

        for di in range(max_len):
            decoder_output, decoder_hidden, attn_w = self.decoder(decoder_input,
                                                                  decoder_hidden,
                                                                  encoder_outputs,
                                                                  mask)
            # decoder_output: (batch_size, target_vocab_size)

            topv, topi = decoder_output.topk(1)
            outputs[di] = decoder_output.to(device)
            teacher_force = random.random() < teacher_forcing_ratio


            decoder_input = target_seq[:, di].unsqueeze(1) if teacher_force else topi.detach()

            if inference: 
#                 print('attn_w', attn_w.size(),attn_w)
                attn_weights.append(attn_w)
    
            if inference and decoder_input == end_token:
                return outputs[:di], attn_weights
            
            

        return outputs, attn_weights
    
    

Initializae model


In [28]:
SEED = 0
BATCH_SIZE = 8

TRAIN_RATIO = 0.8

N = len(thai_romanization_dataset)

print('Number of samples: ', N)
train_split_idx = int(TRAIN_RATIO * N)

print('split at index:', train_split_idx)
indices = list(range(N))

# Random Split
np.random.seed(SEED)
np.random.shuffle(indices)
train_indices, val_indices = indices[:train_split_idx], indices[train_split_idx:]

print('first 5 train indices', train_indices[0:5])
print('first 5 val indices', val_indices[0:5])

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
                                   
train_dataset_loader = torch.utils.data.DataLoader(
                                             thai_romanization_dataset,
                                             batch_size=BATCH_SIZE, 
                                             sampler=train_sampler,
                                             num_workers=0)

val_dataset_loader = torch.utils.data.DataLoader(
                                             thai_romanization_dataset,
                                             batch_size=BATCH_SIZE,
                                             shuffle=False,
                                             sampler=valid_sampler,
                                             num_workers=0)


print('Number of train mini-batches', len(train_dataset_loader))
print('Number of val mini-batches', len(val_dataset_loader))


Number of samples:  14333
split at index: 11466
first 5 train indices [8602, 6753, 8327, 13711, 13348]
first 5 val indices [3244, 2169, 11214, 13832, 12141]
Number of train mini-batches 1434
Number of val mini-batches 359


In [29]:
for i in train_dataset_loader:

    print(i['input_tensor'].shape)
    # print(len(i['input_text']))
    break

torch.Size([8, 70])


In [30]:
INPUT_DIM = len(thai_romanization_dataset.lang_th.char2index)
OUTPUT_DIM = len(thai_romanization_dataset.lang_th_romanized.char2index)

ENC_EMB_DIM = 128
ENC_HID_DIM = 256
ENC_DROPOUT = 0.5

DEC_EMB_DIM = 128
DEC_HID_DIM = 256
DEC_DROPOUT = 0.5

_encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT)
_decoder = AttentionDecoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DEC_DROPOUT)

model = Seq2Seq(_encoder, _decoder).to(device)
model


Seq2Seq(
  (encoder): Encoder(
    (character_embedding): Embedding(75, 128)
    (rnn): LSTM(128, 128, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): AttentionDecoder(
    (character_embedding): Embedding(41, 128)
    (rnn): LSTM(384, 256, batch_first=True)
    (attn): Attn(
      (attn): Linear(in_features=256, out_features=256, bias=True)
    )
    (linear): Linear(in_features=256, out_features=41, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [31]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (character_embedding): Embedding(75, 128)
    (rnn): LSTM(128, 128, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): AttentionDecoder(
    (character_embedding): Embedding(41, 128)
    (rnn): LSTM(384, 256, batch_first=True)
    (attn): Attn(
      (attn): Linear(in_features=256, out_features=256, bias=True)
    )
    (linear): Linear(in_features=256, out_features=41, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [32]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,012,777 trainable parameters


In [33]:
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index = 0)


## Training

In [34]:
print_loss_every = 100
teacher_forcing_ratio = 0.5

def train(model, iterator, optimizer, criterion, clip, teacher_forcing_ratio=0.5):
    
    model.train()
    
    epoch_loss = 0
    for i, batch in tqdm(enumerate(iterator), total = len(iterator)):
        optimizer.zero_grad()

        source_seq, source_seq_len = batch['input_tensor'], batch['input_length']

        batch_size = source_seq.size(0)
        # print('source_seq.size()', source_seq.size())
        assert source_seq.size() == (batch_size, MAX_LENGTH)

        # target_seq: (batch_size , MAX_LENGTH)
        # output: (MAX_LENGTH , batch_size , target_vocab_size)
        target_seq = batch['target_tensor']

        output, _ = model(source_seq, source_seq_len, target_seq, teacher_forcing_ratio=teacher_forcing_ratio)
        
        # target_seq -> (MAX_LENGTH , batch_size)
        target_seq = target_seq.transpose(0, 1)

        # target_seq -> ((MAX_LENGTH) * batch_size)
        target_seq = target_seq[0:].contiguous().view(-1)

        # output -> ((MAX_LENGTH ) * batch_size, target_vocab_size)        
        output = output[0:].view(-1, output.size(-1))

        loss = criterion(output, target_seq)
        
        if i % print_loss_every == 0:
            print('Loss ', loss.item())

        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        
        
    return epoch_loss / len(iterator)

In [35]:
def evaluate(model, iterator, criterion):
    
    model.eval()

    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            source_seq, source_seq_len = batch['input_tensor'], batch['input_length']
            batch_size = source_seq.size(0)

            # target_seq: (batch_size , MAX_LENGTH)
            # output: (MAX_LENGTH , batch_size , target_vocab_size)
            target_seq = batch['target_tensor']
            output, _ = model(source_seq, source_seq_len, target_seq)
        
            # target_seq -> (MAX_LENGTH , batch_size)
            target_seq = target_seq.transpose(0, 1)

            # target_seq -> ((MAX_LENGTH - 1) * batch_size)
            target_seq = target_seq[0:].contiguous().view(-1)

            # output -> ((MAX_LENGTH -1) * batch_size, target_vocab_size)        
            output = output[0:].view(-1, output.size(-1))

            loss = criterion(output, target_seq)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def inference(model, text, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char):
    model.eval()

    input_seq =  [ch for ch in text] +  ['<end>']
    numericalized = [char_2_ix[ch] for ch in input_seq] 
    
#     print('input ',numericalized)
    sentence_length = [len(numericalized)]

    tensor = torch.LongTensor(numericalized).view(1, -1).to(device) 
    
#     print(tensor)
    translation_tensor_logits, attn_weights = model(tensor, sentence_length, None, 0) 
#     print(translation_tensor_logits)
    try:
        translation_tensor = torch.argmax(translation_tensor_logits.squeeze(1), 1).cpu().numpy()
        translation_indices = [t for t in translation_tensor]
        
#         print('translation_tensor', translation_tensor)
        translation = [ix_to_target_char[t] for t in translation_tensor]
    except:
        translation_indices = [0]
        translation = ['<pad>']
    return ''.join(translation), translation_indices, attn_weights

def show_inference_example(model, input_texts, target_texts, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char):
    for index, input_text in enumerate(input_texts):
        prediction, indices, _ = inference(model, input_text, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char)
        print('groundtruth: {}'.format(target_texts[index]))
        print(' prediction: {} {}\n'.format(prediction, indices))

        
def show_inference_example_attn(model, input_texts, target_texts, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char):
    for index, input_text in enumerate(input_texts):
        prediction, indices, attn = inference(model, input_text, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char)
        
        display_attention(input_text, prediction, attn)
        print('groundtruth: {}'.format(target_texts[index]))
        print(' prediction: {} {}\n'.format(prediction, indices))
        
def display_attention(candidate, translation, attention):
    
  
    attns = [attn.squeeze().cpu().detach().numpy() for attn in attention]
#     print(attns)
    x = [t for t in candidate] + ['<end>']

    y = [t for t in translation] + ['<end>']
    y = y
#     print(y)

#     print(attns)
    figsize=(6,6)
    margins = {  #     vvv margin in inches
        "left"   :     1.5 / figsize[0],
        "bottom" :     1.0 / figsize[1],
        "right"  : 1 - 0.3 / figsize[0],
        "top"    : 1 - 1   / figsize[1]
    }
    fig, ax = plt.subplots()

    fig.subplots_adjust(**margins)
    
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,y1 - 100 ,y2 + 100))
    
    sns.heatmap(data=attns, yticklabels=y, cmap='rocket')
    
    fpath = os.path.join(rcParams["datapath"], "/usr/share/fonts/truetype/tlwg/Sawasdee.ttf")
    prop = fm.FontProperties(fname=fpath)

    ax.set_xlabel(x, fontproperties=prop, fontsize=15, labelpad=10)

    ax.set_xticklabels(x, fontproperties=prop, fontsize=15, fontweight=400)
    ax.set_yticklabels(y, fontproperties=prop, rotation = 0, va="center",
                       verticalalignment='top',position=(0,-10))

#     fig.tight_layout(pad=0.0, h_pad=0, w_pad=0)
    plt.tight_layout()

    plt.show()
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
    

In [36]:


char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char = thai_romanization_dataset.lang_th.char2index , \
                                                             thai_romanization_dataset.lang_th.index2char , \
                                                             thai_romanization_dataset.lang_th_romanized.char2index ,\
                                                             thai_romanization_dataset.lang_th_romanized.index2char

In [37]:
#show_inference_example_attn(model, ['บ้านไร่'],  ['banrai'] ,char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char)

    

In [38]:
# print(char_2_ix)

In [39]:
# print(ix_to_target_char)


## Evaluation on val_set with following metrics:
   
1. F1-score (macro-average) -- Character level

2. Exact Match (EM)

3. Exact Match (EM) - Character level

    

In [40]:
# Functions for model performance evaluation
def precision(pred_chars, target_chars):
    # TP / TP + FP
    pred_chars_multiset = Counter(pred_chars)
    target_chars_multiset = Counter(target_chars)

    overlap = list((pred_chars_multiset & target_chars_multiset).elements())
    n_overlap = len(overlap)

    return n_overlap / max(len(pred_chars), 1)

def recall(pred_chars, target_chars):
    # TP / TP + FN
        
    pred_chars_multiset = Counter(pred_chars)
    target_chars_multiset = Counter(target_chars)

    overlap = list((pred_chars_multiset & target_chars_multiset).elements())
    n_overlap = len(overlap)
    return n_overlap / len(target_chars)

def f1(precision, recall):
    
    return (2  * precision * recall) / (precision + recall)

def em(pred, target):
    if pred == target:
        return 1
    return 0

def em_char(pred_chars, target_chars):
    N_target_chars = len(target_chars)
    N_pred_chars = len(pred_chars)

    score = 0
    for index in range(min(N_pred_chars, N_target_chars)):
        if target_chars[index] == pred_chars[index]:
            score+=1
            
    return score / max(N_target_chars, N_pred_chars)

In [41]:
def evaluate_inference(model, val_indices):
    cumulative_precision = 0
    cumulative_recall = 0
    cumulative_em = 0
    cumulative_em_char = 0
    
    N = len(val_indices)

    epoch_loss = 0
    prediction_results = []
    em_char_score = 0
    for i, val_index in tqdm(enumerate(val_indices), total=N):
        input_text = thai_romanization_dataset.input_texts[val_index]
        target_text = thai_romanization_dataset.target_texts[val_index]

        
        prediction, indices, _ = inference(model, input_text, char_2_ix, ix_2_char, target_char_to_ix, ix_to_target_char)
        prediction_results.append(prediction)
        if i <= 15:
            print('Example: {}'.format(i+1))
            print('      input: {}'.format(input_text))
            print('groundtruth: {}'.format(target_text))
            print(' prediction: {}\n'.format(prediction))

        pred_chars = [char for char in prediction]
        target_chars = [char for char in target_text]

        cumulative_precision += precision(pred_chars, target_chars)
        cumulative_recall +=  recall(pred_chars, target_chars)
        cumulative_em_char += em_char(pred_chars, target_chars)
        cumulative_em += em(prediction, target_text)

    macro_average_precision = cumulative_precision / N
    macro_average_recall = cumulative_recall /N
    f1_macro_average = f1(macro_average_precision, macro_average_recall) 
    em_score = cumulative_em / N
    em_char_score = cumulative_em_char / N
    print('')
    print('F1 (macro-average) = ', f1_macro_average)
    print('EM = ', em_score)
    print('EM (Character-level) = ', em_char_score)


    return f1_macro_average, em_score, em_char_score, prediction_results



## Model Training

In [42]:
def writelog(s,ep):
  with open(str(ep)+".txt","w") as f:
    f.write(s)

In [43]:
N_EPOCHS = 50
CLIP = 5

best_f1_score = float('-inf')
teacher_forcing_ratio = 0.4

for epoch in range(0, N_EPOCHS):
    print('epoch={}, teacher_forcing_ratio={}'.format(epoch, teacher_forcing_ratio))
    model.train()
    start_time = time.time()
    train_loss = train(model, train_dataset_loader, optimizer, criterion, CLIP, teacher_forcing_ratio=teacher_forcing_ratio)
    teacher_forcing_ratio = max(teacher_forcing_ratio-0.000, 0.0)
    valid_loss = evaluate(model, val_dataset_loader, criterion)
    if epoch % 1 == 0:
       
        print('Evaluate F1,EM')
        f1_macro_average, em_score, em_char_score, _ = evaluate_inference(model, val_indices[:500])

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if f1_macro_average > best_f1_score:
        best_f1_score = f1_macro_average
        print('save best model em score={} at epoch={}'.format(f1_macro_average, epoch))
        save_model('thai2rom-pytorch.attn.v8', epoch, best_f1_score, model)
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    ss=f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s\n\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}\n\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}'
    ss+="\n"+str(f1_macro_average)+"\t"+str(em_score)+"\t"+str(em_char_score)
    writelog(ss,epoch+1)

epoch=0, teacher_forcing_ratio=0.4


Loss  3.713576316833496
Loss  2.638913869857788
Loss  2.59570574760437
Loss  2.4350507259368896
Loss  2.3910412788391113
Loss  2.258296251296997
Loss  2.210555076599121
Loss  2.099175453186035
Loss  2.0238687992095947
Loss  2.09737229347229
Loss  1.9843709468841553
Loss  1.985033392906189
Loss  1.7946994304656982
Loss  2.4866888523101807
Loss  1.9341843128204346

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: k a                                                                   

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: k a                                                                   

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k a                                                                   

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: k a                                                                   

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: k a                                                                   

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: t a                                                                   

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩

Loss  1.840900182723999
Loss  1.755514144897461
Loss  1.9394631385803223
Loss  2.1437273025512695
Loss  1.7723917961120605
Loss  2.0902132987976074
Loss  1.7715190649032593
Loss  2.083535671234131
Loss  2.1732122898101807
Loss  1.9930672645568848
Loss  1.7175813913345337
Loss  1.5208361148834229
Loss  1.7637752294540405
Loss  1.7300890684127808
Loss  2.2114017009735107

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s a                                                                   

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: k a                                                                   

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: t a                                                                   

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: s a                                                                   

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: s                                                                     

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: t a                                                                   

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩

...

Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s ɯː ˦˥

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: kʰ w aː m ˧ . h oː m ˧ . h oː m ˧ . h oː m ˧ . h oː m ˧ . h oː m ˧ . h

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k l i a̯ ˨˩

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: m oː ˧ . b i k̚ ˨˩

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: pʰ l ɤː ˩˩˦

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: r aː ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩
 prediction: r oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ 

Example: 8
      input: หมากัดอย่ากัดตอบ
groundtruth: m aː ˩˩˦ . k a t̚ ˨˩ . j aː ˨˩ . k a t̚ ˨˩ . t ɔː p̚ ˨˩
 prediction: m aː k̚ ˨˩ . t aː k̚ ˨˩ 

Loss  0.7016505599021912
Loss  0.9903337359428406
Loss  0.8071762323379517
Loss  0.7333003878593445
Loss  0.9312527775764465
Loss  0.6966783404350281
Loss  0.7532159090042114
Loss  0.44695189595222473
Loss  0.49474775791168213
Loss  0.798778235912323
Loss  0.35346612334251404
Loss  0.6914264559745789
Loss  1.047414779663086
Loss  0.9244154095649719
Loss  0.7725073099136353

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s ɯː ˦˥

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . 

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k l i a̯ ˨˩

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: m oː ˧ . b i k̚ ˨˩

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: pʰ l ɤː ˩˩˦

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: r aː ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t͡ɕʰ a  ˧ . t

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩
 prediction: r oː ˧ . m a k̚ ˦˥ . m a k̚ ˦˥ . m a k̚ ˦˥ . m a k̚ ˦˥ . m a k̚ ˦˥ . m

Example: 8
      input: หมากัดอย่ากัดตอบ
groundtruth: m aː ˩˩˦ . k a t̚ ˨˩ . j aː ˨˩ . k a t̚ ˨˩ . t ɔː p̚ ˨˩
 prediction: m aː t̚ ˨˩ . t aː t̚ ˨˩ 

Loss  0.525652289390564
Loss  0.8995144367218018
Loss  0.8822696805000305
Loss  1.1395471096038818
Loss  1.0482672452926636
Loss  0.7360109090805054
Loss  0.7834205031394958
Loss  0.8142841458320618
Loss  0.9552593231201172
Loss  1.1881111860275269
Loss  0.9404150247573853
Loss  0.94515061378479
Loss  1.106825590133667
Loss  0.8364823460578918
Loss  0.8986330628395081

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s ɯː ˦˥

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: kʰ w aː m ˧ . h oː m ˧ . h oː m ˧ . h oː m ˧ . h oː m ˧ . h oː m ˧ . h

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k l i a̯ ˨˩

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: m oː ˧ . k i m ˧ . k i m ˧ . k i m ˧ . k i m ˧ . k i m ˧ . k i m ˧ . k

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: pʰ l ɤː ˩˩˦

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: r aː ˧ . s aː ˧ . s aː ˧ . s aː ˧ . s aː ˧ . s aː ˧ . s aː ˧ . s aː ˧ 

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩
 prediction: r oː ˧ . m ɔː ˧ . m ɔː ˧ . m ɔː ˧ . m ɔː ˧ . m ɔː ˧ . m ɔː ˧ . m ɔː ˧ 

Example: 8
      input: หมากัดอย่ากัดตอบ
groundtruth: m aː ˩˩˦ . k a t̚ ˨˩ . j aː ˨˩ . k a t̚ ˨

In [44]:
print("hi")

hi


In [45]:
save_model('thai2rom-pytorch.attn.v8', epoch, em_score, model)

Save model at epoch  49


In [46]:
N_EPOCHS = 100

In [ ]:
for epoch in range(50, N_EPOCHS):
    print('epoch={}, teacher_forcing_ratio={}'.format(epoch, teacher_forcing_ratio))
    model.train()
    start_time = time.time()
    train_loss = train(model, train_dataset_loader, optimizer, criterion, CLIP, teacher_forcing_ratio=teacher_forcing_ratio)
    teacher_forcing_ratio = max(teacher_forcing_ratio-0.000, 0.0)
    valid_loss = evaluate(model, val_dataset_loader, criterion)
    if epoch % 1 == 0:
       
        print('Evaluate F1,EM')
        f1_macro_average, em_score, em_char_score, _ = evaluate_inference(model, val_indices[:500])

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if f1_macro_average > best_f1_score:
        best_f1_score = f1_macro_average
        print('save best model em score={} at epoch={}'.format(f1_macro_average, epoch))
        save_model('thai2rom-pytorch.attn.v8', epoch, best_f1_score, model)
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    ss=f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s\n\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}\n\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}'
    ss+="\n"+str(f1_macro_average)+"\t"+str(em_score)+"\t"+str(em_char_score)
    writelog(ss,epoch+1)

epoch=50, teacher_forcing_ratio=0.4


Loss  0.8459103107452393
Loss  0.5849997401237488
Loss  0.4545709192752838
Loss  0.6307719349861145
Loss  0.8062809109687805
Loss  0.6429563164710999
Loss  0.7586973309516907
Loss  0.6918904781341553
Loss  0.6388946175575256
Loss  0.819267213344574
Loss  0.7395625710487366
Loss  0.6983305215835571
Loss  0.7494703531265259
Loss  0.9781633615493774
Loss  1.0958049297332764

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s ɯː ˦˥

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: k oː m ˧ . k oː m ˧ . k oː m ˧ . k oː m ˧ . k oː m ˧ . k oː m ˧ . k oː

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k l i a̯ ˨˩

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: m oː ˧ . b i k̚ ˨˩

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: pʰ l ɤː ˩˩˦

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: r aː ˧ . t͡ɕʰ a p̚ ˨˩

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩
 prediction: r oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ 

Example: 8
      input: หมากัดอย่ากัดตอบ
groundtruth: m aː ˩˩˦ . k a t̚ ˨˩ . j aː ˨˩ . k a t̚ ˨˩ . t ɔː p̚ ˨˩
 prediction: m aː t̚ ˨˩ . t aː t̚ ˨˩ . t aː t̚ ˨˩ . t aː t̚ ˨˩ . t aː t̚ ˨˩ . t aː 

E

Loss  0.6865025758743286
Loss  0.8767294883728027
Loss  0.8364272713661194
Loss  0.7753809094429016
Loss  0.8412173390388489
Loss  1.1746081113815308
Loss  0.7440217733383179
Loss  0.5678955316543579
Loss  0.6653854846954346
Loss  0.6999717950820923
Loss  0.5792884826660156
Loss  0.8647432923316956
Loss  0.3519381582736969
Loss  0.6342419385910034
Loss  0.9127464890480042

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s ɯː ˦˥

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . 

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k l i a̯ ˨˩

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: m oː ˧ . b i k̚ ˨˩

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: pʰ l ɤː ˩˩˦

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: r aː ˧ . s a  t̚ ˨˩

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩
 prediction: r oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ . l oː ˧ 

Example: 8
      input: หมากัดอย่ากัดตอบ
groundtruth: m aː ˩˩˦ . k a t̚ ˨˩ . j aː ˨˩ . k a t̚ ˨˩ . t ɔː p̚ ˨˩
 prediction: m aː k̚ ˨˩ . t aː k̚ ˨˩ . t aː k̚ ˨˩ . t aː k̚ ˨˩ . t aː k̚ ˨˩ . t aː 

Exa

...

Loss  0.5477880239486694
Loss  0.6738989949226379
Loss  0.5018103122711182
Loss  0.9207618832588196
Loss  0.6743404865264893
Loss  0.5401330590248108
Loss  0.4998283088207245
Loss  0.8225631713867188
Loss  0.44199642539024353
Loss  0.8974811434745789
Loss  0.6366018056869507
Loss  0.5700506567955017
Loss  0.9897115230560303
Loss  0.4844246506690979
Loss  0.6657188534736633

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s ɯː ˦˥

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . kʰ w aː m ˧ . 

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k l i a̯ ˨˩

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: m oː ˧ . b i k̚ ˨˩

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: pʰ l ɤː ˩˩˦

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: r aː ˧ . s a p̚ ˨˩

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩
 prediction: r oː ˧ . n a k̚ ˦˥

Example: 8
      input: หมากัดอย่ากัดตอบ
groundtruth: m aː ˩˩˦ . k a t̚ ˨˩ . j aː ˨˩ . k a t̚ ˨˩ . t ɔː p̚ ˨˩
 prediction: m aː k̚ ˨˩ . k aː ˩ . k aː ˩ . k aː ˩ . k aː ˩ . k aː ˩ . k aː ˩ . k a

Example: 9
      input: ญิบ
groundtruth: j i p̚ ˦˥
 pred

Loss  0.713051438331604
Loss  0.7512778639793396
Loss  0.4297276735305786
Loss  0.7884737253189087
Loss  0.9548162221908569
Loss  0.944734513759613
Loss  0.7905272245407104
Loss  0.7193683981895447
Loss  1.2577502727508545
Loss  0.627566933631897
Loss  0.7670553922653198
Loss  0.7673878073692322
Loss  0.48404091596603394
Loss  0.8330340385437012
Loss  0.8200972080230713

Evaluate F1,EM


Example: 1
      input: ซื้อ
groundtruth: s ɯː ˦˥
 prediction: s ɯː ˦˥

Example: 2
      input: ความโกหก
groundtruth: kʰ w aː m ˧ . k oː ˧ . h o k̚ ˨˩
 prediction: kʰ w aː m ˧ . h oː ˧ . h oː ˧ . h oː ˧ . h oː ˧ . h oː ˧ . h oː ˧ . h 

Example: 3
      input: เกลี่ย
groundtruth: k l i a̯ ˨˩
 prediction: k l i a̯ ˨˩

Example: 4
      input: โมซัมบิก
groundtruth: m oː ˧ . s a m ˧ . b i k̚ ˨˩
 prediction: m oː ˧ . k i k̚ ˨˩

Example: 5
      input: เผลอ
groundtruth: pʰ l ɤː ˩˩˦
 prediction: pʰ . l l ɤː ˩˩˦

Example: 6
      input: ราชาศัพท์
groundtruth: r aː ˧ . t͡ɕʰ aː ˧ . s a p̚ ˨˩
 prediction: r aː ˧ . t͡ɕʰ a p̚ ˨˩

Example: 7
      input: โรมันคาทอลิก
groundtruth: r oː ˧ . m a n ˧ . kʰ aː ˧ . tʰ ɔː ˧ . l i k̚ ˨˩
 prediction: r oː ˧ . m a k̚ ˦˥

Example: 8
      input: หมากัดอย่ากัดตอบ
groundtruth: m aː ˩˩˦ . k a t̚ ˨˩ . j aː ˨˩ . k a t̚ ˨˩ . t ɔː p̚ ˨˩
 prediction: m aː k̚ ˨˩ . k aː k̚ ˨˩ . k aː k̚ ˨˩ . k aː k̚ ˨˩ . k aː k̚ ˨˩ . k aː 

Example: 9
      input: ญิบ
groundtruth: j i p̚ ˦

Loss  0.4085376262664795
Loss  0.7721119523048401
Loss  0.5740694999694824
Loss  1.0038424730300903
Loss  0.8267717361450195
Loss  0.5170999765396118
Loss  0.6572713851928711
Loss  1.0326344966888428
Loss  1.0198661088943481
Loss  0.6574210524559021
Loss  0.857706606388092
Loss  0.683279275894165
Loss  0.6996129155158997
Loss  0.6109996438026428
Loss  0.8653448224067688

